# Finally running our model on Edge TPU

So, we are finally ready to run our model on the Edge TPU:

In [1]:
from edgetpu.classification.engine import ClassificationEngine
from tensorflow.lite.python.interpreter import Interpreter
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

First, let's recover the quantization scale information (you can just save this info beforehand):

In [2]:
def get_quant_info(model_path='./data/mnist_cnn_quant.tflite'):
    with open(model_path, 'rb') as f:
        model_content = f.read()
        
    interpreter = Interpreter(model_content=bytes(model_content))

    input_details = interpreter.get_input_details()
    input_scale, input_zero_point = input_details[0]['quantization']
    input_stats = (input_scale, input_zero_point)

    return input_stats

input_stats = get_quant_info('./data/mnist_cnn_quant.tflite')

In [3]:
engine = ClassificationEngine('data/mnist_cnn_quant_edgetpu.tflite')

Define the inference process:

In [4]:
def run_inference(data, engine, input_stats):
    input_scale, input_zero_point = input_stats    
    
    quantized_data = np.uint8(data/input_scale + input_zero_point)
    
    predicted_class = engine.ClassifyWithInputTensor(quantized_data, top_k=1)
    
    return predicted_class[0][0]

Measure the performance of the deployed classifier:

In [6]:
mnist = input_data.read_data_sets('MNIST-data', one_hot=True)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


Let's send an image to the Edge TPU:

In [7]:
quantized_data = np.uint8(mnist.test.images[0]/input_stats[0] + input_stats[1])
predicted_class = engine.ClassifyWithInputTensor(quantized_data, top_k=1)

In [8]:
print(predicted_class)

[(7, 0.99609375)]


In [9]:
predictions = [run_inference(x, engine, input_stats) for x in mnist.test.images]

In [10]:
predictions = np.asarray(predictions)
true_labels = mnist.test.labels.argmax(1)
accuracy = np.sum(predictions == true_labels)/len(true_labels)
print("Accuracy :", 100*accuracy, "%")


Accuracy : 99.3 %
